<a href="https://colab.research.google.com/github/W-Sebastian/JupyterNotebooks/blob/master/Structuri_U%C8%99oare_Grind%C4%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analiza grindei sandwich: studiu de caz pentru o trambulină

## Introducere

## Definirea problemei

Se propune pentru analiză studiul asupra unei trambuline de piscină de tip structură sandwich. Plecăm de la următoarea schiță:
![schita trambulina](img/schita.png)

Modelul are următoarele constrângeri:
- Materialele folosite pentru înveliș respectic miez sunt date;
- Lungimea grinzii trebuie să fie între 1 și 4 metri;
- Se impune o rigiditate a grinzii de maxim 5 N/mm;
- Un capăt de grindă este considerat încastrat;
- Capătul liber are o condiție la limită definită prin masă de 150 Kg și accelerație de 9.834 m/s^2;
- Lățimea grinzii este de 500 mm;
- Factorul de siguranță pentru calculul tensiunilor este 5;
- Costul materialelor pentru construcția grinzii nu trebuie să depășească 200€; prețul pentru fiecare material este dar în €/Kg.

Se propune alegerea materialelor pentru înveliș și miez și alegerea lungimii respectiv a grosimii miezului și învelișului astfel încât:
- Să nu se treacă de costul impus;
- Să se reducă masa cât mai mult.

Din datele problemei ne vom propune următoarele obiective:
- Definirea modelelor analitice de calcul;
- Corelarea rezultatelor analitice cu o analiză de element finit;
- Găsirea combinației care ne oferă cea mai mică masă respectând limitările date;
- Adițional vom explora și găsirea celui mai mic preț care să respecte limitările date;

## Considerente limitative

- Toate valorile din document sunt ținute în MKS (meter/kilogram/second) pentru a păsta uniformitatea calculelor și a reduce potențiale probleme legate de conversia de unit-uri. Conversile din alte unităti (mm, MPa) sunt făcut înainte sau după calculele propriu-zise. Pentru presiuni vom folosi unitatea Pa.
- Se consideră în calcule o grindă cu înveliș exterior aplicat doar deasupra și sub grindă, fără a modela marginile ei deoarece le vom considera neglijabile;
- Pe cât posibil se vor folosi abrevierile din formule pentru concepte alăturate de comentarii care vor explica semnificația lor;
- În cod, comentarile, explicațiile și numele simbolurilor sunt în Engleza; explicațile din document sunt în Română;
- Se vor aplica simplificări la modelele matematice conform teoriei grinzilor sandwich.

# Definirea elementelor de bază pentru calculele analitice

Pentru a simplifica restul aplicației vom începe prin a defini o serie de structuri care să modeleze conceptele din modelul analitic.
Începem prin a defini structurile pentru parametrii de material, pentru geometria grinzii și pentru simularea statică.

Vom defini 2 structuri diferite materiale, una va modela parametrii materialelor de înveliș iar cealalta parametrii materialelor de miez.  
Pentru materialele de înveliș avem nevoie de:
- Densitate ($\rho$);
- Modulul elastic longitudinal ($E_f$);
- Tensiunea maximă admsibilă ($\tau_{af}$);
- Costul materialului în €/Kg.

Pentru materialele de miez avem nevoie de:
- Densitate ($\rho$);
- Modulul elastic longitudinal ($E_f$);
- Modulul elastic transversal ($G_c$)
- Tensiunea maximă admsibilă ($\tau_{ac}$);
- Costul materialului în €/Kg.

In [10]:
class SkinMaterialParameters:
    def __init__(self, rho, Ef, tau_af, Cost):
        self.rho = rho
        self.Ef = Ef # Longitudinal Elastic Modulus
        self.tau_af = tau_af # Persmissible Shear Stress
        self.Cost = Cost # euro/kg

class CoreMaterialParameters:
    def __init__(self, rho, Ec, Gc, tau_ac, Cost):
        self.rho = rho
        self.Ec = Ec # Longitudinal Elastic Modulus
        self.Gc = Gc # Transversal Elastic Modulus
        self.tau_ac = tau_ac # Permissible Shear Stress
        self.Cost = Cost

Pentru geometria grinzii avem nevoie de următorii parametrii:

- Materialul pentru înveliș;
- Materialul pentru miez;
- Lungimea ($L$);
- Lățimea ($b$);
- Grosimea miezului ($tc$);
- Grosimea învelișului ($tf$);

In [11]:
class BeamModel:
    def __init__(self, L, b, tf, tc, skinMat, coreMat):
        self.L = L # Lenght
        self.b = b # Width
        self.tf = tf # Skin height
        self.tc = tc # Core height
        self.SkinMat = skinMat # Material for the skin
        self.CoreMat = coreMat # Material for the core

Ramâne să definim parametrii pentru simularea modelului analitic. Aici vom utiliza următoarele date:
- Masa aplicată la capătul liber al grinzii ($m$);
- Accelerația folosită pentru aplicarea condiției la limită ($a$);
- Rigiditatea maximă admisă în grindă ($k_m$);
- Modelul de grindă (instanță a clasei `BeamModel`);
- Factorul de siguranță folosit în calculele tensiunilor ($s_f$).

In [12]:
class BeamSimulation:
    def __init__(self, m, a, km, model, sf):
        self.m = m
        self.a = a
        self.km = km
        self.model = model
        self.sf = sf

Cu aceste clase putem modela complet o simulare a unei grinzi. 

## Definirea propietătilor de material admise

Adăugăm, conform specificaților impuse, propietățile de materiale.

In [13]:
from enum import Enum

class SkinMaterials(Enum):
    Steel = 1
    Aluminium = 2
    GFRP = 3
    CFRP = 4

class CoreMaterials(Enum):
    DivinycellH60 = 1
    DivinycellH100 = 2
    DivinycellH130 = 3
    DivinycellH200 = 4

skin_materials = {
    SkinMaterials.Steel: SkinMaterialParameters(7800, 205000 * 1e6, 300 * 1e6, 0.4),
    SkinMaterials.Aluminium: SkinMaterialParameters(2700, 70000 * 1e6, 200 * 1e6, 0.7),
    SkinMaterials.GFRP: SkinMaterialParameters(1600, 20000 * 1e6, 400 * 1e6, 4),
    SkinMaterials.CFRP: SkinMaterialParameters(2700, 70000 * 1e6, 1000 * 1e6, 80)
    }
core_materials = {
    CoreMaterials.DivinycellH60: CoreMaterialParameters(60, 550 * 1e6, 22 * 1e6, 0.6 * 1e6, 6),
    CoreMaterials.DivinycellH100: CoreMaterialParameters(100, 95 * 1e6, 38 * 1e6, 1.2 * 1e6, 10),
    CoreMaterials.DivinycellH130: CoreMaterialParameters(130, 125 * 1e6, 47 * 1e6, 1.6 * 1e6, 13),
    CoreMaterials.DivinycellH200: CoreMaterialParameters(200, 195 * 1e6, 75 * 1e6, 3.0 * 1e6, 20),
    }



In [14]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

labels = ['Material', 'E (N/mm)', 'G (N/mm)', '$tau$ (MPa)', 'Cost (€/kg)']
values = ['test', 1, 2, 3, 4]

fig = go.Figure(data=[ go.Table(header=dict(values=labels), cells=dict(values=values))])
fig.show()

ModuleNotFoundError: No module named 'plotly'